In [ ]:
import pandas as pd
import tkinter as tk
from  tkinter import filedialog, messagebox
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

def select_output_file(title, file_types):
    """File selection dialog for output"""
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    
    try:
        file_path = filedialog.asksaveasfilename(
            title=title,
            filetypes=file_types,
            defaultextension=file_types[0][1]
        )
    finally:
        root.destroy()
    
    return file_path if file_path else None

def get_state_coordinates():
    """
    Returns a dictionary with US state capitals and centroids
    """
    # US States data with capitals and approximate centroids
    states_data = {
        'Alabama': {'capital': ('Montgomery', 32.3792, -86.3077), 'centroid': (32.7794, -86.8287)},
        'Alaska': {'capital': ('Juneau', 58.3019, -134.4197), 'centroid': (64.2008, -149.4937)},
        'Arizona': {'capital': ('Phoenix', 33.4484, -112.0740), 'centroid': (34.2744, -111.6602)},
        'Arkansas': {'capital': ('Little Rock', 34.7465, -92.2896), 'centroid': (34.8938, -92.4426)},
        'California': {'capital': ('Sacramento', 38.5816, -121.4944), 'centroid': (37.2502, -119.7512)},
        'Colorado': {'capital': ('Denver', 39.7392, -104.9903), 'centroid': (39.0146, -105.5471)},
        'Connecticut': {'capital': ('Hartford', 41.7658, -72.6734), 'centroid': (41.6219, -72.7273)},
        'District of Columbia': {'capital': ('Washington', 38.9072, -77.0369), 'centroid': (38.9072, -77.0369)},
        'Delaware': {'capital': ('Dover', 39.1582, -75.5244), 'centroid': (38.9896, -75.5050)},
        'Florida': {'capital': ('Tallahassee', 30.4383, -84.2807), 'centroid': (28.6305, -82.4497)},
        'Georgia': {'capital': ('Atlanta', 33.7490, -84.3880), 'centroid': (32.6415, -83.4426)},
        'Hawaii': {'capital': ('Honolulu', 21.3069, -157.8583), 'centroid': (20.2927, -156.3737)},
        'Idaho': {'capital': ('Boise', 43.6150, -116.2023), 'centroid': (44.3509, -114.6130)},
        'Illinois': {'capital': ('Springfield', 39.7817, -89.6501), 'centroid': (40.0417, -89.1965)},
        'Indiana': {'capital': ('Indianapolis', 39.7684, -86.1581), 'centroid': (39.8942, -86.2816)},
        'Iowa': {'capital': ('Des Moines', 41.5868, -93.6250), 'centroid': (42.0751, -93.4960)},
        'Kansas': {'capital': ('Topeka', 39.0473, -95.6752), 'centroid': (38.4937, -98.3804)},
        'Kentucky': {'capital': ('Frankfort', 38.2009, -84.8733), 'centroid': (37.5347, -85.3021)},
        'Louisiana': {'capital': ('Baton Rouge', 30.4515, -91.1871), 'centroid': (31.1801, -92.0877)},
        'Maine': {'capital': ('Augusta', 44.3107, -69.7795), 'centroid': (45.3695, -69.2428)},
        'Maryland': {'capital': ('Annapolis', 38.9784, -76.4922), 'centroid': (39.0458, -76.6413)},
        'Massachusetts': {'capital': ('Boston', 42.3601, -71.0589), 'centroid': (42.2373, -71.5314)},
        'Michigan': {'capital': ('Lansing', 42.7325, -84.5555), 'centroid': (44.3467, -85.4102)},
        'Minnesota': {'capital': ('St. Paul', 44.9537, -93.0900), 'centroid': (46.2807, -94.3053)},
        'Mississippi': {'capital': ('Jackson', 32.2988, -90.1848), 'centroid': (32.7541, -89.6678)},
        'Missouri': {'capital': ('Jefferson City', 38.5767, -92.1735), 'centroid': (38.3566, -92.4580)},
        'Montana': {'capital': ('Helena', 46.5891, -112.0391), 'centroid': (47.0527, -109.6333)},
        'Nebraska': {'capital': ('Lincoln', 40.8136, -96.7026), 'centroid': (41.5378, -99.7951)},
        'Nevada': {'capital': ('Carson City', 39.1638, -119.7674), 'centroid': (39.3289, -116.6312)},
        'New Hampshire': {'capital': ('Concord', 43.2081, -71.5376), 'centroid': (43.6805, -71.5811)},
        'New Jersey': {'capital': ('Trenton', 40.2206, -74.7597), 'centroid': (40.1907, -74.6728)},
        'New Mexico': {'capital': ('Santa Fe', 35.6870, -105.9378), 'centroid': (34.4071, -106.1126)},
        'New York': {'capital': ('Albany', 42.6526, -73.7562), 'centroid': (42.9538, -75.5268)},
        'North Carolina': {'capital': ('Raleigh', 35.7796, -78.6382), 'centroid': (35.5557, -79.3877)},
        'North Dakota': {'capital': ('Bismarck', 46.8083, -100.7837), 'centroid': (47.4501, -100.4659)},
        'Ohio': {'capital': ('Columbus', 39.9612, -82.9988), 'centroid': (40.2862, -82.7937)},
        'Oklahoma': {'capital': ('Oklahoma City', 35.4676, -97.5164), 'centroid': (35.5889, -97.4943)},
        'Oregon': {'capital': ('Salem', 44.9429, -123.0351), 'centroid': (43.9336, -120.5583)},
        'Pennsylvania': {'capital': ('Harrisburg', 40.2732, -76.8867), 'centroid': (40.8781, -77.7996)},
        'Rhode Island': {'capital': ('Providence', 41.8240, -71.4128), 'centroid': (41.5801, -71.5317)},
        'South Carolina': {'capital': ('Columbia', 34.0007, -81.0348), 'centroid': (33.8569, -80.9450)},
        'South Dakota': {'capital': ('Pierre', 44.3683, -100.3510), 'centroid': (44.4443, -100.2263)},
        'Tennessee': {'capital': ('Nashville', 36.1627, -86.7816), 'centroid': (35.8580, -86.3505)},
        'Texas': {'capital': ('Austin', 30.2672, -97.7431), 'centroid': (31.4757, -99.3312)},
        'Utah': {'capital': ('Salt Lake City', 40.7608, -111.8910), 'centroid': (39.3210, -111.0937)},
        'Vermont': {'capital': ('Montpelier', 44.2601, -72.5754), 'centroid': (44.0687, -72.6658)},
        'Virginia': {'capital': ('Richmond', 37.5407, -77.4360), 'centroid': (37.5215, -78.8537)},
        'Washington': {'capital': ('Olympia', 47.0379, -122.9007), 'centroid': (47.3826, -120.4472)},
        'West Virginia': {'capital': ('Charleston', 38.3498, -81.6326), 'centroid': (38.6409, -80.6227)},
        'Wisconsin': {'capital': ('Madison', 43.0731, -89.4012), 'centroid': (44.6243, -89.9941)},
        'Wyoming': {'capital': ('Cheyenne', 41.1400, -104.8202), 'centroid': (42.9957, -107.5512)}
    }
    return states_data

def calculate_distances():
    """Calculate distances between US states and Saint Lucia"""
    # Saint Lucia coordinates (capital: Castries and approximate centroid)
    saint_lucia = {
        'capital': (14.0101, -60.9875),  # Castries
        'centroid': (13.9094, -60.9789)  # Approximate centroid
    }
    
    states_data = get_state_coordinates()
    distances = []
    
    for state, coords in states_data.items():
        # Calculate distance from state capital to Saint Lucia capital
        capital_distance = geodesic(
            (coords['capital'][1], coords['capital'][2]),  # state capital coords
            saint_lucia['capital']  # Saint Lucia capital coords
        ).miles
        
        # Calculate distance from state centroid to Saint Lucia centroid
        centroid_distance = geodesic(
            coords['centroid'],  # state centroid coords
            saint_lucia['centroid']  # Saint Lucia centroid coords
        ).miles
        
        distances.append({
            'State': state,
            'State Capital': coords['capital'][0],
            'Capital to Capital Distance (miles)': round(capital_distance, 2),
            'Centroid to Centroid Distance (miles)': round(centroid_distance, 2)
        })
    
    return pd.DataFrame(distances)

def main():
    try:
        print("Calculating distances...")
        distances_df = calculate_distances()
        
        # Sort by Capital to Capital distance
        distances_df = distances_df.sort_values('Capital to Capital Distance (miles)')
        
        # Get output file location
        excel_types = [
            ('Excel files', '*.xlsx'),
            ('All files', '*.*')
        ]
        
        output_file = select_output_file(
            title='Save Distance Calculations As',
            file_types=excel_types
        )
        
        if not output_file:
            print("No output location selected. Exiting.")
            return
        
        # Ensure proper file extension
        if not output_file.endswith('.xlsx'):
            output_file += '.xlsx'
        
        # Save to Excel with formatted output
        print("Saving results...")
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            distances_df.to_excel(writer, sheet_name='Distances', index=False)
            
            # Auto-adjust column widths
            worksheet = writer.sheets['Distances']
            for idx, col in enumerate(distances_df.columns):
                max_length = max(
                    distances_df[col].astype(str).apply(len).max(),
                    len(col)
                )
                worksheet.column_dimensions[chr(65 + idx)].width = max_length + 2
        
        print(f"Distance calculations saved to: {output_file}")
        messagebox.showinfo("Success", f"Distance calculations have been saved to:\n{output_file}")
        
    except Exception as e:
        error_msg = f"An error occurred:\n{str(e)}"
        print(error_msg)
        messagebox.showerror("Error", error_msg)

if __name__ == "__main__":
    main()